<a href="https://colab.research.google.com/github/pandov/myitacademy-pandov/blob/master/FERFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext tensorboard
num_classes = 5
# ! sh colab.sh setup
# ! dvc pull -d stages/BIOMETRY{num_classes}.dvc
# ! unzip "data/external/*.zip"
# ! dvc repro stages/BIOMETRY{num_classes}.dvc

In [ ]:
# %%capture
import torch
# from catalyst.dl.utils import set_global_seed, prepare_cudnn
# from src.kcv import CrossValidation, Experiment, experiment_run
# from src.utils import get_fer_names, get_fc_fer_weights
# from src.callbacks import *
from src.features import BIOMETRY
from src.models import FERFC
# set_global_seed(7)
# prepare_cudnn(deterministic=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# weight = get_fc_fer_weights(num_classes).to(device)
from torchsummary import summary
model = FERFC(5).to(device)
summary(model, input_size=(1, 163))

In [ ]:
class Ex1(Experiment):
    def model(self): return FERFC(num_classes=num_classes, hidden_size=256)
    def criterion(self): return torch.nn.CrossEntropyLoss(weight=weight)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    def scheduler(self, optimizer): return torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
class Ex2(Ex1):
    def model(self): return FERFC(num_classes=num_classes, hidden_size=512)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
class Ex3(Ex1):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-5)
class Ex4(Ex2):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)

class Ex5(Ex3):
    def model(self): return FERFC(num_classes=num_classes, hidden_size=1024)
class Ex6(Ex4):
    def model(self): return FERFC(num_classes=num_classes, hidden_size=1024)

In [ ]:
%tensorboard --logdir logs

In [ ]:
%%capture
experiments = [Ex1(), Ex2(), Ex3(), Ex4(), Ex5(), Ex6()]
class_names = get_fer_names(num_classes)
callbacks = [
    AccuracyCallback(num_classes=num_classes),
    AUCCallback(num_classes=num_classes, class_names=class_names),
    PrecisionRecallF1ScoreCallback(num_classes=num_classes, class_names=class_names),
    ConfusionMatrixCallback(num_classes=num_classes, class_names=class_names),
]
cross_val = CrossValidation(BIOMETRY(num_classes), 4)
logdir = f'logs/FERFC{num_classes}'
experiment_run(experiments, callbacks, cross_val, logdir, device, 'f1/_mean', 100, 250)

In [ ]:
! sh colab.sh save $logdir

In [ ]:
# ! rm -r logs/FERFC5